In [12]:
# Get Snowpark components
import os
from snowflake.snowpark import Session

In [13]:
# Create connection string
connection_parameters = {
"account":  '' ,
 "user":  '' ,
 "password":  '' ,
 "role":  '' ,
 "warehouse":  '' ,
 "database":  'COALESCE_DEV' ,
 "schema": 'FIFA_PREDICTION'
}

session = Session.builder.configs(connection_parameters).create()

In [14]:
# Install needed packages
# Be sure check that packages also exist in Acaconda repo for Snowflake
!pip install scipy numpy seaborn pandas statsmodels 

In [15]:
import pandas as pd
import numpy as np
import seaborn
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import poisson
from snowflake.snowpark.functions import udf

In [16]:
# Let's fetch training data that we have in Snowflake about FIFA games
# Data is limited to games played after 2000 for validity reasons and also because this limits the size of the model
df = session.sql("SELECT * FROM RESULTS WHERE TO_DATE(DATE, 'YYYY-MM-DD') > TO_DATE('2000-01-01', 'YYYY-MM-DD')").to_pandas()

In [17]:
# It seems that score values are casted as strings, let's cast them to floats
df = df[['HOME_TEAM','AWAY_TEAM','HOME_SCORE','AWAY_SCORE']]
df['HOME_SCORE'] = df['HOME_SCORE'].astype(float)
df['AWAY_SCORE'] = df['AWAY_SCORE'].astype(float)

In [18]:
# Lets build the model using Poisson model
goal_model_data = pd.concat([df[['HOME_TEAM','AWAY_TEAM','HOME_SCORE']].assign(home=1).rename(
            columns={'HOME_TEAM':'team', 'AWAY_TEAM':'opponent','HOME_SCORE':'goals'}),
           df[['AWAY_TEAM','HOME_TEAM','AWAY_SCORE']].assign(home=0).rename(
            columns={'AWAY_TEAM':'team', 'HOME_TEAM':'opponent','AWAY_SCORE':'goals'})])

model = smf.glm(formula="goals ~ home + team + opponent", data=goal_model_data, 
                        family=sm.families.Poisson()).fit()

In [19]:
# Lets import the model to Snowflake stage
session.sql('create or replace stage MODELSTAGE').collect() #Create a model stage if it does not already exist. 

[Row(status='Stage area MODELSTAGE successfully created.')]

In [20]:
# Let's build the Snowflake Python UDF using Snowpark @udf command
# Necessary packages are imported in the same command
# This is part is needed only once

@udf(name='predict_result',is_permanent = True, stage_location = '@MODELSTAGE', replace=True, session=session, packages=["numpy", "pandas", "scipy", "statsmodels"])
def simulate_match(homeTeam: str, awayTeam: str) -> float:
    home_goals_avg = model.predict(pd.DataFrame(data={'team': homeTeam, 
                                                            'opponent': awayTeam,'home':1},
                                                      index=[1])).values[0]
    away_goals_avg = model.predict(pd.DataFrame(data={'team': awayTeam, 
                                                            'opponent': homeTeam,'home':0},
                                                      index=[1])).values[0]
    team_pred = [[poisson.pmf(i, team_avg) for i in range(0, 10)] for team_avg in [home_goals_avg, away_goals_avg]]
    home_win=(np.outer(np.array(team_pred[0]), np.array(team_pred[1])))
    result = np.sum(np.tril(home_win, -1))
    return result
